In [1]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score


In [2]:
df = pd.read_csv('data/bulidata.csv')
df

,Unnamed: 0,MATCH_DATE,LEAGUE_NAME,SEASON,LEAGUE,FINISHED,LOCATION,VIEWER,MATCHDAY,MATCHDAY_NR,...,HOME_ICON,AWAY_TEAM_ID,AWAY_TEAM_NAME,AWAY_TEAM,AWAY_ICON,GOALS_HOME,GOALS_AWAY,DRAW,WIN_HOME,WIN_AWAY
0,0,2005-08-05 20:30:00,1. Fussball-Bundesliga 2005/2006,2005,bl1,True,München,NaN,1. Spieltag,1,...,https://i.imgur.com/jJEsJrj.png,87,Borussia Mönchengladbach,Gladbach,https://i.imgur.com/KSIk0Eu.png,3,0,0.0,1.0,0.0
1,1,2005-08-06 15:30:00,1. Fussball-Bundesliga 2005/2006,2005,bl1,True,Köln,NaN,1. Spieltag,1,...,https://upload.wikimedia.org/wikipedia/en/thum...,81,1. FSV Mainz 05,Mainz,https://upload.wikimedia.org/wikipedia/commons...,1,0,0.0,1.0,0.0
2,2,2005-08-06 15:30:00,1. Fussball-Bundesliga 2005/2006,2005,bl1,True,Duisburg,NaN,1. Spieltag,1,...,https://upload.wikimedia.org/wikipedia/en/c/c8...,16,VfB Stuttgart,Stuttgart,https://i.imgur.com/v0tkpNx.png,1,1,1.0,0.0,0.0
3,3,2005-08-06 15:30:00,1. Fussball-Bundesliga 2005/2006,2005,bl1,True,Hamburg,NaN,1. Spieltag,1,...,https://upload.wikimedia.org/wikipedia/commons...,79,1. FC Nürnberg,Nürnberg,https://upload.wikimedia.org/wikipedia/commons...,3,0,0.0,1.0,0.0
4,4,2005-08-06 15:30:00,1. Fussball-Bundesliga 2005/2006,2005,bl1,True,Wolfsburg,NaN,1. Spieltag,1,...,https://i.imgur.com/ucqKV4B.png,7,Borussia Dortmund,BVB,https://upload.wikimedia.org/wikipedia/commons...,2,2,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5503,5505,2023-05-27 15:30:00,1. Fußball-Bundesliga 2022/2023,2022,bl1,True,Köln,NaN,34. Spieltag,34,...,https://upload.wikimedia.org/wikipedia/en/thum...,40,FC Bayern München,Bayern,https://i.imgur.com/jJEsJrj.png,1,2,0.0,0.0,1.0
5504,5506,2023-05-27 15:30:00,1. Fußball-Bundesliga 2022/2023,2022,bl1,True,Leipzig,NaN,34. Spieltag,34,...,https://i.imgur.com/Rpwsjz1.png,9,FC Schalke 04,Schalke,https://upload.wikimedia.org/wikipedia/commons...,4,2,0.0,1.0,0.0
5505,5507,2023-05-27 15:30:00,1. Fußball-Bundesliga 2022/2023,2022,bl1,True,Stuttgart,NaN,34. Spieltag,34,...,https://i.imgur.com/v0tkpNx.png,175,TSG 1899 Hoffenheim,Hoffenheim,https://i.imgur.com/gF0PfEl.png,1,1,1.0,0.0,0.0
5506,5508,2023-05-27 15:30:00,1. Fußball-Bundesliga 2022/2023,2022,bl1,True,Berlin,NaN,34. Spieltag,34,...,https://assets.dfb.de/uploads/000/018/232/smal...,134,Werder Bremen,Bremen,https://upload.wikimedia.org/wikipedia/commons...,1,0,0.0,1.0,0.0


In [3]:

df.dtypes

Unnamed: 0          int64
MATCH_DATE         object
LEAGUE_NAME        object
SEASON              int64
LEAGUE             object
FINISHED             bool
LOCATION           object
VIEWER            float64
MATCHDAY           object
MATCHDAY_NR         int64
HOME_TEAM_ID        int64
HOME_TEAM_NAME     object
HOME_TEAM          object
HOME_ICON          object
AWAY_TEAM_ID        int64
AWAY_TEAM_NAME     object
AWAY_TEAM          object
AWAY_ICON          object
GOALS_HOME          int64
GOALS_AWAY          int64
DRAW              float64
WIN_HOME          float64
WIN_AWAY          float64
dtype: object

In [4]:
df = df.drop(columns=['LEAGUE_NAME', 'LEAGUE', 'FINISHED', 'LOCATION', 'VIEWER', 'MATCHDAY', 'HOME_TEAM', 'HOME_ICON', 'AWAY_TEAM', 'AWAY_ICON'])
df = df.drop(columns=['MATCH_DATE','Unnamed: 0'])
df['MATCH_RESULT'] = (df['WIN_HOME'].eq(1).astype(int) * 1 + 
                      df['WIN_AWAY'].eq(1).astype(int) * 0 + 
                      df['DRAW'].eq(1).astype(int) * 2)
df.drop(columns=['WIN_HOME', 'WIN_AWAY', 'DRAW'], inplace=True)
home_team_dict = dict(zip(df['HOME_TEAM_NAME'], df['HOME_TEAM_ID']))
df = df.drop(columns=['HOME_TEAM_NAME', 'AWAY_TEAM_NAME'])

In [5]:
df_2023 = pd.read_csv('data/2023_blstats.csv')
df_2023 = df_2023.drop(columns=['Match Number', 'Location', 'Date'])
df_2023['SEASON'] = 2023
tbc = ('SV Werder Bremen', 'Werder Bremen'), ('TSG Hoffenheim','TSG 1899 Hoffenheim'), ('Bayer 04 Leverkusen','Bayer Leverkusen'), ('VfL Bochum 1848', 'VfL Bochum'), ('Sport-Club Freiburg', 'SC Freiburg'), ('1. FC Heidenheim 1846','989')
tbc_dict = dict(tbc)
df_2023['Home Team'] = df_2023['Home Team'].replace(tbc_dict)
df_2023['Away Team'] = df_2023['Away Team'].replace(tbc_dict)
df_2023['HOME_TEAM_ID'] = df_2023['Home Team'].replace(home_team_dict)
df_2023['AWAY_TEAM_ID'] = df_2023['Away Team'].replace(home_team_dict)
df_2023 = df_2023.drop(columns=['Away Team', 'Home Team'])
df_2023['HOME_TEAM_ID'] = df_2023['HOME_TEAM_ID'].astype(int)
df_2023['AWAY_TEAM_ID'] = df_2023['AWAY_TEAM_ID'].astype(int)
df_2023

,Round Number,Result,SEASON,HOME_TEAM_ID,AWAY_TEAM_ID
0,1,0 - 4,2023,134,40
1,1,5 - 0,2023,16,129
2,1,4 - 4,2023,95,87
3,1,1 - 2,2023,175,112
4,1,2 - 0,2023,131,989
...,...,...,...,...,...
301,34,1 - 3,2023,131,81
302,34,2 - 2,2023,91,1635
303,34,2 - 1,2023,6,95
304,34,2 - 1,2023,80,112


In [6]:
df_2023[['GOALS_HOME', 'GOALS_AWAY']] = df_2023['Result'].str.split('-', expand=True).astype(int)
df_2023.drop(columns=['Result'], inplace=True)
df_2023.rename(columns={'Round Number': 'MATCHDAY_NR'}, inplace=True)

conditions = [
    (df_2023['GOALS_HOME'] > df_2023['GOALS_AWAY']),
    (df_2023['GOALS_HOME'] == df_2023['GOALS_AWAY']),
    (df_2023['GOALS_HOME'] < df_2023['GOALS_AWAY'])
]
results = [1, 2, 0]

df_2023['MATCH_RESULT'] = np.select(conditions, results)
df_2023

,MATCHDAY_NR,SEASON,HOME_TEAM_ID,AWAY_TEAM_ID,GOALS_HOME,GOALS_AWAY,MATCH_RESULT
0,1,2023,134,40,0,4,0
1,1,2023,16,129,5,0,1
2,1,2023,95,87,4,4,2
3,1,2023,175,112,1,2,0
4,1,2023,131,989,2,0,1
...,...,...,...,...,...,...,...
301,34,2023,131,81,1,3,0
302,34,2023,91,1635,2,2,2
303,34,2023,6,95,2,1,1
304,34,2023,80,112,2,1,1


In [7]:
home_team_dict["1. FC Heidenheim 1846"] = 989
print(sorted(home_team_dict))

['1. FC Heidenheim 1846', '1. FC Kaiserslautern', '1. FC Köln', '1. FC Nürnberg', '1. FC Union Berlin', '1. FSV Mainz 05', 'Alemannia Aachen', 'Arminia Bielefeld', 'Bayer Leverkusen', 'Borussia Dortmund', 'Borussia Mönchengladbach', 'Eintracht Braunschweig', 'Eintracht Frankfurt', 'FC Augsburg', 'FC Bayern München', 'FC Energie Cottbus', 'FC Hansa Rostock', 'FC Ingolstadt 04', 'FC Schalke 04', 'FC St. Pauli', 'Fortuna Düsseldorf', 'Hamburger SV', 'Hannover 96', 'Hertha BSC', 'Karlsruher SC', 'MSV Duisburg', 'RB Leipzig', 'SC Freiburg', 'SC Paderborn 07', 'SV Darmstadt 98', 'SpVgg Greuther Fürth', 'TSG 1899 Hoffenheim', 'VfB Stuttgart', 'VfL Bochum', 'VfL Wolfsburg', 'Werder Bremen']


In [8]:
test_df = pd.read_csv('data/bundesliga_2024_25.csv')
test_df = test_df.dropna(axis=1, how='all')

non_nan_values = test_df['Unnamed: 3'].dropna()

# Display the non-NaN values
non_nan_values

test_df['HOME'] = test_df['HOME'].fillna(test_df['Unnamed: 3'])
test_df['AWAY'] = test_df['AWAY'].fillna(test_df['Unnamed: 6'])

home_team_dict["Holstein Kiel"] = 834
test_df['HOME_TEAM_ID'] = test_df['HOME'].replace(home_team_dict)
test_df['AWAY_TEAM_ID'] = test_df['AWAY'].replace(home_team_dict)
test_df.drop(columns=['Unnamed: 3', 'Unnamed: 6', 'HOME', 'AWAY'], inplace=True)

test_df 

/tmp/ipykernel_7847/764317454.py:13: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  test_df['HOME_TEAM_ID'] = test_df['HOME'].replace(home_team_dict)
/tmp/ipykernel_7847/764317454.py:14: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  test_df['AWAY_TEAM_ID'] = test_df['AWAY'].replace(home_team_dict)


,MATCHDAY_NR,HOME_TEAM_ID,AWAY_TEAM_ID
0,1,112,6
1,1,1635,129
2,1,7,91
3,1,175,834
4,1,112,16
...,...,...,...
300,34,112,91
301,34,95,80
302,34,81,6
303,34,87,131


In [9]:
#Training and testing the data in the 23/24 season
train = df[(df['SEASON'] <= 2022) & (df['SEASON'] >= 2006)]
test = df_2023

rf = RandomForestClassifier(n_estimators=100, min_samples_split= 20, random_state=20, max_depth=10)
predictors = ['HOME_TEAM_ID', 'AWAY_TEAM_ID', 'MATCHDAY_NR']
rf.fit(train[predictors], train['MATCH_RESULT'])
predictions = rf.predict(test[predictors])
accuracy_score = accuracy_score(test['MATCH_RESULT'], predictions)
accuracy_score

0.5130718954248366

In [10]:
combined = pd.DataFrame(dict(actual=test['MATCH_RESULT'], predicted=predictions))
pd.crosstab(index = combined['actual'], columns=combined['predicted'])

predicted,0,1,2
actual,,,
0,39,41,11
1,10,106,18
2,14,55,12


In [11]:
def calculate_points(future, team_id):
    home_games = future[future['HOME_TEAM_ID'] == team_id]
    away_games = future[future['AWAY_TEAM_ID'] == team_id]

    home_count = home_games['PREDICTION'].value_counts()
    away_count = away_games['PREDICTION'].value_counts()

    points = (
        home_count.get(1, 0) * 3 + 
        home_count.get(2, 0) + 
        away_count.get(0, 0) * 3 + 
        away_count.get(2, 0)
    )
    return points

future = test_df
train = pd.concat([df, df_2023], axis=0)
train = train[(train['SEASON'] == 2023) | (train['SEASON'] == 2013)]
rf.fit(train[predictors], train['MATCH_RESULT'])
predictions = rf.predict(future[predictors])
future['PREDICTION'] = predictions

# Get unique team IDs
team_ids = pd.concat([future['HOME_TEAM_ID'], future['AWAY_TEAM_ID']]).unique()

# Calculate points for each team
points_dict = {team_id: calculate_points(future, team_id) for team_id in team_ids}

# Create points table
points_table = pd.DataFrame(list(points_dict.items()), columns=['Team_ID', 'Points'])

# Sort points table by points in descending order
points_table = points_table.sort_values(by='Points', ascending=False).reset_index(drop=True)

team_id_to_name = {v: k for k, v in home_team_dict.items()}

# Map team IDs to team names
points_table['Team_Name'] = points_table['Team_ID'].map(team_id_to_name)
points_table = points_table[['Team_Name', 'Team_ID', 'Points']]

print(points_table)

                   Team_Name  Team_ID  Points
0           Bayer Leverkusen        6     100
1          Borussia Dortmund        7      94
2          FC Bayern München       40      81
3              VfB Stuttgart       16      72
4                 RB Leipzig     1635      69
5      1. FC Heidenheim 1846      989      48
6              Holstein Kiel      834      48
7        TSG 1899 Hoffenheim      175      43
8            1. FSV Mainz 05       81      38
9                FC Augsburg       95      36
10                VfL Bochum      129      35
11  Borussia Mönchengladbach       87      35
12       Eintracht Frankfurt       91      34
13             VfL Wolfsburg      131      33
14               SC Freiburg      112      29
15             Werder Bremen      134      28
16              FC St. Pauli       98      28
17        1. FC Union Berlin       80      26
